In [2]:
%pip install pandas numpy jsonlines
#Imports
import pandas as pd
import numpy as np
import random
import jsonlines

Note: you may need to restart the kernel to use updated packages.


In [3]:

# load data
df_data = pd.read_csv('/Users/abhi/GitHUB/FederatedRAG/DataSets/ml-100k/u.data', delimiter='\t',header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
df_user = pd.read_csv('/Users/abhi/GitHUB/FederatedRAG/DataSets/ml-100k/u.user', delimiter='|', header=None, names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])
df_item = pd.read_csv('/Users/abhi/GitHUB/FederatedRAG/DataSets/ml-100k/u.item', delimiter='|', encoding='latin-1', header=None, names=['movieid', 'movietitle', 'releasedate', 'videoreleasedate', 'imdburl', 'unknown', 'action', 'adventure', 'animation', "children's", 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film-noir', 'horror', 'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western'])


In [4]:
#Constants
PER_USER_DATAPOINTS = 6
MOVIES_IN_DATAPOINTS = 5
TYPE_MOVIE = {'Good', 'Bad',}


In [5]:
df_data

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [6]:
df_user

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [7]:
df_item

,movieid,movietitle,releasedate,videoreleasedate,imdburl,unknown,action,adventure,animation,children's,...,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_data['user_id'].value_counts()

user_id
405    737
655    685
13     636
450    540
276    518
      ... 
441     20
36      20
812     20
895     20
93      20
Name: count, Length: 943, dtype: int64

In [9]:
df_data['item_id'].value_counts()

item_id
50      583
258     509
100     508
181     507
294     485
       ... 
852       1
1505      1
1653      1
1452      1
1641      1
Name: count, Length: 1682, dtype: int64

In [10]:
df_item.columns

Index(['movieid', 'movietitle', 'releasedate', 'videoreleasedate', 'imdburl',
       'unknown', 'action', 'adventure', 'animation', 'children's', 'comedy',
       'crime', 'documentary', 'drama', 'fantasy', 'film-noir', 'horror',
       'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war',
       'western'],
      dtype='object')

In [11]:
#Generate signle datapoint

def gen_datapoint(userId, df_data,df_item):
    #get all the movies that the user has rated
    user_ratings = df_data[df_data['user_id'] == userId]
    random_user_segment = random.choice(list(TYPE_MOVIE))
    datapoint = []
    if random_user_segment == 'Good':
        user_ratings = user_ratings[user_ratings['rating'] > 3.5]
    else:
        user_ratings = user_ratings[user_ratings['rating'] < 2.5]
    # print(user_ratings)
    for i in range(PER_USER_DATAPOINTS):
        #get a random movie that the user has rated
        movie = user_ratings.sample()
        movie_id = movie['item_id'].values[0]
        movie_rating = movie['rating'].values[0]
        movie_title = df_item[df_item['movieid'] == movie_id]['movietitle'].values[0]
        movie_genre = 'unknown'
        for x in ['unknown', 'action', 'adventure', 'animation', "children's", 'comedy',
       'crime', 'documentary', 'drama', 'fantasy', 'film-noir', 'horror',
       'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war',
       'western']:
            if df_item[df_item['movieid'] == movie_id][x].iloc[0]==1:
                movie_genre = x
        datapoint.append([userId, movie_id, movie_rating, movie_title, movie_genre])
        # print(userId, movie_id, movie_rating, movie_title)
    return (random_user_segment, datapoint)


In [12]:
gen_datapoint(df_data['user_id'].sample().values[0], df_data, df_item)

('Bad',
 [[561, 231, 2, 'Batman Returns (1992)', 'crime'],
  [561, 588, 2, 'Beauty and the Beast (1991)', 'musical'],
  [561, 141, 2, '20,000 Leagues Under the Sea (1954)', 'sci-fi'],
  [561, 550, 1, 'Die Hard: With a Vengeance (1995)', 'thriller'],
  [561, 678, 2, 'Volcano (1997)', 'thriller'],
  [561, 1110, 2, 'Tank Girl (1995)', 'sci-fi']])

In [13]:
new_line = '\n'
def prompt(data):
    rating, datapoints = data
    c_prompt = f"Given a user, here is the list of movies and their genres that they have rated: "
    for datapoint in datapoints:
        c_prompt += f"{new_line}Movie: {datapoint[3]}, Genre: {datapoint[4]}, Rating: {datapoint[2]}"
    return c_prompt


In [14]:
a = ('Good',
 [[315, 520, 4, 'Great Escape, The (1963)', 'war'],
  [315, 180, 4, 'Apocalypse Now (1979)', 'war'],
  [315, 285, 5, 'Secrets & Lies (1996)', 'drama'],
  [315, 520, 4, 'Great Escape, The (1963)', 'war'],
  [315, 216, 4, 'When Harry Met Sally... (1989)', 'romance'],
  [315, 303, 4, "Ulee's Gold (1997)", 'drama']])
print(prompt(a))

Given a user, here is the list of movies and their genres that they have rated: 
Movie: Great Escape, The (1963), Genre: war, Rating: 4
Movie: Apocalypse Now (1979), Genre: war, Rating: 4
Movie: Secrets & Lies (1996), Genre: drama, Rating: 5
Movie: Great Escape, The (1963), Genre: war, Rating: 4
Movie: When Harry Met Sally... (1989), Genre: romance, Rating: 4
Movie: Ulee's Gold (1997), Genre: drama, Rating: 4


In [15]:
rating, datapoints = a
result = {'rating': rating, 'movie1': datapoints[0][3], 'movie2': datapoints[1][3], 'movie3': datapoints[2][3], 'movie4': datapoints[3][3], 'movie5': datapoints[4][3],
          'genre1': datapoints[0][4], 'genre2': datapoints[1][4], 'genre3': datapoints[2][4], 'genre4': datapoints[3][4], 'genre5': datapoints[4][4]}
result

{'rating': 'Good',
 'movie1': 'Great Escape, The (1963)',
 'movie2': 'Apocalypse Now (1979)',
 'movie3': 'Secrets & Lies (1996)',
 'movie4': 'Great Escape, The (1963)',
 'movie5': 'When Harry Met Sally... (1989)',
 'genre1': 'war',
 'genre2': 'war',
 'genre3': 'drama',
 'genre4': 'war',
 'genre5': 'romance'}

In [16]:
from tqdm import tqdm


unique_user_ids = df_user['user_id'].unique().tolist()

data_list = []

# Loop through the ratings
for userId in tqdm(unique_user_ids, desc='Generating prompts'):
    # Call the prompt function
    try:
        if len(df_data[df_data['user_id']==userId])>40:
            rating, datapoints = gen_datapoint(userId, df_data,df_item)
            data_list.append({'rating': rating, 'movie1': datapoints[0][3], 'movie2': datapoints[1][3], 'movie3': datapoints[2][3], 'movie4': datapoints[3][3], 'movie5': datapoints[4][3], 'movie6': datapoints[5][3],
                                            'genre1': datapoints[0][4], 'genre2': datapoints[1][4], 'genre3': datapoints[2][4], 'genre4': datapoints[3][4], 'genre5': datapoints[4][4], 'genre6': datapoints[5][4]})
            rating, datapoints = gen_datapoint(userId, df_data,df_item)
            data_list.append({'rating': rating, 'movie1': datapoints[0][3], 'movie2': datapoints[1][3], 'movie3': datapoints[2][3], 'movie4': datapoints[3][3], 'movie5': datapoints[4][3], 'movie6': datapoints[5][3],
                                            'genre1': datapoints[0][4], 'genre2': datapoints[1][4], 'genre3': datapoints[2][4], 'genre4': datapoints[3][4], 'genre5': datapoints[4][4], 'genre6': datapoints[5][4]})
            
        if len(df_data[df_data['user_id']==userId])>20:
            rating, datapoints = gen_datapoint(userId, df_data,df_item)
            data_list.append({'rating': rating, 'movie1': datapoints[0][3], 'movie2': datapoints[1][3], 'movie3': datapoints[2][3], 'movie4': datapoints[3][3], 'movie5': datapoints[4][3], 'movie6': datapoints[5][3],
                                            'genre1': datapoints[0][4], 'genre2': datapoints[1][4], 'genre3': datapoints[2][4], 'genre4': datapoints[3][4], 'genre5': datapoints[4][4], 'genre6': datapoints[5][4]})
            
        rating, datapoints = gen_datapoint(userId, df_data,df_item)
        data_list.append({'rating': rating, 'movie1': datapoints[0][3], 'movie2': datapoints[1][3], 'movie3': datapoints[2][3], 'movie4': datapoints[3][3], 'movie5': datapoints[4][3], 'movie6': datapoints[5][3],
                                            'genre1': datapoints[0][4], 'genre2': datapoints[1][4], 'genre3': datapoints[2][4], 'genre4': datapoints[3][4], 'genre5': datapoints[4][4], 'genre6': datapoints[5][4]})
    except:
        continue

    


Generating prompts: 100%|██████████| 943/943 [04:37<00:00,  3.40it/s]


In [17]:
data_list

[{'rating': 'Good',
  'movie1': 'Terminator 2: Judgment Day (1991)',
  'movie2': 'Star Trek: The Wrath of Khan (1982)',
  'movie3': 'Disclosure (1994)',
  'movie4': 'Hudsucker Proxy, The (1994)',
  'movie5': 'Mask, The (1994)',
  'movie6': 'Mighty Aphrodite (1995)',
  'genre1': 'thriller',
  'genre2': 'sci-fi',
  'genre3': 'thriller',
  'genre4': 'romance',
  'genre5': 'fantasy',
  'genre6': 'comedy'},
 {'rating': 'Bad',
  'movie1': 'Heavy Metal (1981)',
  'movie2': 'Homeward Bound: The Incredible Journey (1993)',
  'movie3': 'Bad Boys (1995)',
  'movie4': 'Homeward Bound: The Incredible Journey (1993)',
  'movie5': 'All Dogs Go to Heaven 2 (1996)',
  'movie6': 'Mimic (1997)',
  'genre1': 'sci-fi',
  'genre2': "children's",
  'genre3': 'action',
  'genre4': "children's",
  'genre5': 'musical',
  'genre6': 'thriller'},
 {'rating': 'Good',
  'movie1': 'Priest (1994)',
  'movie2': 'Empire Strikes Back, The (1980)',
  'movie3': 'Fugitive, The (1993)',
  'movie4': 'Apollo 13 (1995)',
  'mov

In [18]:
len(data_list)

3089

In [19]:
training_data = data_list[:int(len(data_list)/3)*2]
validation_data = data_list[int(len(data_list)/3)*2:]

In [20]:
def create_jsonl_file(datapoints, file_path):
    with jsonlines.open(file_path, mode='w') as writer:
        for datapoint in datapoints:
            writer.write(datapoint)

file_path = '/Users/abhi/GitHUB/FederatedRAG/DataSets/training.jsonl'
create_jsonl_file(training_data, file_path)

file_path = '/Users/abhi/GitHUB/FederatedRAG/DataSets/validation.jsonl'
create_jsonl_file(validation_data, file_path)


In [21]:
df_data_g = df_data.groupby('item_id')['rating'].mean().reset_index()
df_data_g

,item_id,rating
0,1,3.878319
1,2,3.206107
2,3,3.033333
3,4,3.550239
4,5,3.302326
...,...,...
1677,1678,1.000000
1678,1679,3.000000
1679,1680,2.000000
1680,1681,3.000000


In [22]:
df_movie_rating = df_data_g.merge(df_item[['movieid','movietitle']], how = 'left', left_on = 'item_id', right_on = 'movieid')

In [23]:
df_movie_rating

,item_id,rating,movieid,movietitle
0,1,3.878319,1,Toy Story (1995)
1,2,3.206107,2,GoldenEye (1995)
2,3,3.033333,3,Four Rooms (1995)
3,4,3.550239,4,Get Shorty (1995)
4,5,3.302326,5,Copycat (1995)
...,...,...,...,...
1677,1678,1.000000,1678,Mat' i syn (1997)
1678,1679,3.000000,1679,B. Monkey (1998)
1679,1680,2.000000,1680,Sliding Doors (1998)
1680,1681,3.000000,1681,You So Crazy (1994)


In [24]:
df_movie_rating['rating'] = df_movie_rating['rating'].astype(int)

In [25]:
df_movie_rating[['movieid','movietitle', 'rating']].to_csv("movie_ratings.csv", index = False)

In [1]:
import pandas as pd
import numpy as np

# Load the datasets
ratings = pd.read_csv('/Users/abhi/GitHUB/FederatedRAG/DataSets/ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv('/Users/abhi/GitHUB/FederatedRAG/DataSets/ml-100k/u.item', sep='|', encoding='latin-1', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

# Merge the datasets on movie_id
merged_data = pd.merge(ratings, movies[['movie_id', 'movie_title']], on='movie_id')

# Calculate the average rating per movie and round up to the nearest integer
average_ratings = merged_data.groupby('movie_id').agg({'rating': 'mean', 'movie_title': 'first'}).reset_index()
average_ratings['rating'] = np.ceil(average_ratings['rating']).astype(int)

# Save the result to a new CSV file
average_ratings.to_csv('/Users/abhi/GitHUB/FederatedRAG/DataSets/movie_ratings.csv', index=False)
# Extract genres
genres = ['unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies['genres'] = movies[genres].apply(lambda x: ','.join(x.index[x == 1].tolist()), axis=1)

# Merge the datasets on movie_id and include genres
merged_data_with_genres = pd.merge(ratings, movies[['movie_id', 'movie_title', 'genres']], on='movie_id')

# Calculate the average rating per movie and round up to the nearest integer
average_ratings_with_genres = merged_data_with_genres.groupby('movie_id').agg({'rating': 'mean', 'movie_title': 'first', 'genres': 'first'}).reset_index()
average_ratings_with_genres['rating'] = np.ceil(average_ratings_with_genres['rating']).astype(int)

# Save the result to a new CSV file
average_ratings_with_genres.to_csv('/Users/abhi/GitHUB/FederatedRAG/DataSets/KG_dataset.csv', index=False)